# Introduction
This is the Marginal Effects at Means portion of the project
Please be sure to run DataCleaning.ipynb first to prepare the data

This notebook based on Fastai V1 ML course

## Imports
Import libraries and write settings here.

In [247]:
from fastai.tabular import *
from fastai import *

# Data manipulation
import pandas as pd
import numpy as np

# Options for pandas
pd.options.display.max_columns = 60
pd.options.display.max_rows = 60

# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from IPython import get_ipython
ipython = get_ipython()

# autoreload extension
if 'autoreload' not in ipython.extension_manager.loaded:
    %load_ext autoreload

%autoreload 2

# Visualizations
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)

import cufflinks as cf
cf.go_offline(connected=True)
cf.set_config_file(theme='white')

# Load train/test
If either of the following fail then please be sure to run DataCleaning.ipynb first to prepare the data

In [248]:
# first column in each file is the dependent variable, the rest are independant
csv_files=['vote12gop','TrumpGEVote','TrumpPVote']

whichfle=2
fn = csv_files[whichfle]

jsonfile = 'tmp/' + fn + '_catcont.json'

outdir = 'outBElectionResultsMEMs'
filename='results.csv'
filename_all = 'results_complete.csv'
filename_model_params = "RF_model_params.sav"
os.makedirs(outdir, exist_ok=True)

#get continuous and categorical variables
with open(jsonfile) as f:
    data1 = json.load(f)
    res_cont = data1[0]
    res_cat = data1[1] 
    filter= data1[3]
    
trnfile = 'tmp/' + fn +'_'+filter+'_train'
tstfile = 'tmp/' + fn +'_'+filter+'_tst'


In [249]:
trnfile
tstfile
filter
fn

'tmp/TrumpPVote_cc.inddum_train'

'tmp/TrumpPVote_cc.inddum_tst'

'cc.inddum'

'TrumpPVote'

In [250]:
#get processed training and test data
trn = pd.read_feather(trnfile)
tst = pd.read_feather(tstfile)
# columns_dep_var= 'cc.vote16'

In [251]:
len(trn)

5948

In [252]:
trn.drop('index',axis=1,inplace=True);
tst.drop('index',axis=1,inplace=True);

## Get continuous and categorical variables, convert dependant variable to int64

In [253]:
columns_dep_var = trn.columns[0]
columns_dep_var

'cc.trumppvote'

In [254]:
#remove the dependant variable
res_cont = [x for x in res_cont if x not in columns_dep_var]
res_cat = [x for x in res_cat if x not in columns_dep_var]

In [255]:
trn.head()

,cc.trumppvote,cc.age,cc.blackdum,cc.cc16_304,cc.cc16_305_2,cc.cc16_422c,cc.cc16_422d,cc.cc16_422e,cc.cc16_422f,cc.catholic,cc.evanprot,cc.ideo7,cc.inddum,cc.religiosity,cc.repdum,cc.sex,cc.whitedum,cc.emp.nojob,cc.faminc,cc.i.white.educhs,cc.immviewsum,cc.maxeduc.4yr,cc.union,crashpc,demo.popdense,job.uer,mort.ucd.despair.disc95.pdpy,rustpc
0,0,0.689612,1,0.631982,0,0.517074,0.659101,0.818020,0.386066,0,0,0.225539,1,0.543035,0,1,0,0,1.124369,0,0.000000,0,0,0.954196,0.159130,0.767272,0.785942,0.911288
1,0,1.201895,0,0.947973,1,1.551223,0.988651,0.409010,1.158199,0,1,1.127693,1,1.267082,0,0,1,0,1.405461,0,1.275854,0,0,0.271458,0.047007,0.730736,1.020814,0.001882
2,1,1.103379,0,0.947973,0,0.517074,0.988651,1.636039,0.772133,0,0,0.902155,1,0.362024,0,0,1,0,0.983823,0,0.425285,1,0,0.201719,0.084833,0.507861,0.880143,0.255189
3,0,1.398927,0,0.631982,0,0.517074,0.329550,1.636039,0.386066,0,0,0.451077,1,1.086071,0,1,1,0,1.686553,0,0.000000,1,0,0.383675,0.040013,0.741697,1.058572,0.199546
4,0,1.261005,0,0.631982,0,1.034148,0.329550,0.409010,0.386066,0,0,0.676616,1,1.086071,0,0,1,0,1.264915,0,0.425285,0,0,0.069955,0.063948,0.801982,0.834638,0.000000


In [256]:
#convert trumpgevote to long (otherwise fit fails)
trn[columns_dep_var] = trn[columns_dep_var].astype('int64');
tst[columns_dep_var] = tst[columns_dep_var].astype('int64');

print(str(len(trn)))
print(str(len(tst)))

5948
661


In [257]:
#split out trn_y and tst_y
#this is the dep_var, converted to an int
trn_y = trn[columns_dep_var].copy()
tst_y = tst[columns_dep_var].copy()
trn_y.astype('int64');
trn_y.astype('int64');

trn.drop(columns_dep_var,axis=1,inplace=True);
tst.drop(columns_dep_var,axis=1,inplace=True);

## Lets see what features are corelated with each other

In [258]:
# from rfpimp import plot_corr_heatmap
# viz = plot_corr_heatmap(trn, figsize=(50,30))
# viz.view()

## Categorify and Fill Missing

In [259]:
#from docs https://docs.fast.ai/tabular.transform.html
# tfm = Categorify(cat_names=res_cat, cont_names=res_cont)
# tfm(trn)
# tfm(tst)
# #just checking to see if it works on any old variable
# trn[res_cat[0]].cat.categories
# tst[res_cat[0]].cat.categories

In [260]:
tfm1 = FillMissing(cat_names=res_cat, cont_names=res_cont, add_col=False)
tfm1(trn)
tfm1(tst)

## Clean up any missing columns that result from unfortunate test selection

In [261]:
columns_dep_var

'cc.trumppvote'

In [262]:
#You cannot have any Nan (missing data) fields or random forest will not work.
print(f"Total trn columns = {len(trn.columns)}, total tst columns = {len(tst.columns)}") 
print(f"Total trn columns with Nans= {len(trn.columns[trn.isnull().any()])}") #add ~ to get columns with no missing values
print(f"Total tst columns with Nans= {len(tst.columns[tst.isnull().any()])}") #add ~ to get columns with no missing values

Total trn columns = 27, total tst columns = 27
Total trn columns with Nans= 0
Total tst columns with Nans= 0


In [263]:
#hmmm if either has 1 more column than train then see what it is
set(tst.columns)-set(trn.columns)
set(trn.columns)-set(tst.columns)

#missing one of the _na columns.  This is added, and set to 1, when a variable has an NaN value to mark
#columns that have NaNs

#find the index of the column in trn
# idx=trn.columns.tolist().index('cc.catholic_na')
# idx
# type(trn.columns)

# tst.insert(loc=idx, column='cc.catholic_na', value=False)

set()

set()

In [264]:
trn.head()

,cc.age,cc.blackdum,cc.cc16_304,cc.cc16_305_2,cc.cc16_422c,cc.cc16_422d,cc.cc16_422e,cc.cc16_422f,cc.catholic,cc.evanprot,cc.ideo7,cc.inddum,cc.religiosity,cc.repdum,cc.sex,cc.whitedum,cc.emp.nojob,cc.faminc,cc.i.white.educhs,cc.immviewsum,cc.maxeduc.4yr,cc.union,crashpc,demo.popdense,job.uer,mort.ucd.despair.disc95.pdpy,rustpc
0,0.689612,1,0.631982,0,0.517074,0.659101,0.818020,0.386066,0,0,0.225539,1,0.543035,0,1,0,0,1.124369,0,0.000000,0,0,0.954196,0.159130,0.767272,0.785942,0.911288
1,1.201895,0,0.947973,1,1.551223,0.988651,0.409010,1.158199,0,1,1.127693,1,1.267082,0,0,1,0,1.405461,0,1.275854,0,0,0.271458,0.047007,0.730736,1.020814,0.001882
2,1.103379,0,0.947973,0,0.517074,0.988651,1.636039,0.772133,0,0,0.902155,1,0.362024,0,0,1,0,0.983823,0,0.425285,1,0,0.201719,0.084833,0.507861,0.880143,0.255189
3,1.398927,0,0.631982,0,0.517074,0.329550,1.636039,0.386066,0,0,0.451077,1,1.086071,0,1,1,0,1.686553,0,0.000000,1,0,0.383675,0.040013,0.741697,1.058572,0.199546
4,1.261005,0,0.631982,0,1.034148,0.329550,0.409010,0.386066,0,0,0.676616,1,1.086071,0,0,1,0,1.264915,0,0.425285,0,0,0.069955,0.063948,0.801982,0.834638,0.000000


# Train a RandomForest on all data

In [265]:
trn.head().columns
len(trn.columns)

Index(['cc.age', 'cc.blackdum', 'cc.cc16_304', 'cc.cc16_305_2', 'cc.cc16_422c',
       'cc.cc16_422d', 'cc.cc16_422e', 'cc.cc16_422f', 'cc.catholic',
       'cc.evanprot', 'cc.ideo7', 'cc.inddum', 'cc.religiosity', 'cc.repdum',
       'cc.sex', 'cc.whitedum', 'cc.emp.nojob', 'cc.faminc',
       'cc.i.white.educhs', 'cc.immviewsum', 'cc.maxeduc.4yr', 'cc.union',
       'crashpc', 'demo.popdense', 'job.uer', 'mort.ucd.despair.disc95.pdpy',
       'rustpc'],
      dtype='object')

27

In [266]:
from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display
from sklearn import metrics

#create a random forest object
m_rf = RandomForestClassifier(n_estimators=100, n_jobs=-1, oob_score=True, max_features='auto', min_samples_leaf=10)

In [267]:

def rmse(x,y): 
    '''this and R**2 used for continuous variables'''
    return math.sqrt(((x-y)**2).mean())

def print_score(m, trn, trn_y, tst, tst_y):
    '''
    
    '''
    res = [m.score(trn, trn_y), m.score(tst, tst_y)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [268]:
#train the random forest 
m_rf.fit(trn, trn_y)

print_score(m_rf, trn, trn_y, tst, tst_y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=10, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=True, random_state=None, verbose=0, warm_start=False)

[0.8703765971755212, 0.794251134644478, 0.7995965030262273]


# Test the model on test data

In [269]:
# len(tst.columns)

In [270]:
def eval_accuracy(preds,targs, silent=True):
    totals = len(preds)
    matches = 0
    for x in zip(preds,targs):
        if x[0]==x[1]:
            matches+=1
    acc=100*matches/totals 
    if( silent == False):
        print(f"Got {matches} right out of {totals} samples, Accuracy is {acc} percent")
    return acc

In [271]:
preds1 = m_rf.predict(tst)

eval_accuracy(preds1,tst_y,silent=False);

Got 525 right out of 661 samples, Accuracy is 79.4251134644478 percent


## Save this model for later

In [272]:
# save the model to disk
pickle.dump(m_rf, open(outdir+"/"+filename_model_params, 'wb'))
 
# load the model from disk
m_rf = pickle.load(open(outdir+"/"+filename_model_params, 'rb'))

# Now run MEMs on columns of interest

In [273]:
#this directory contains symlink created at command line like this
# ln -s ../Marginal_Effects_at_Means ./Marginal_Effects_at_Means
#it allows this directory to find Marginal_Effects_at_Means, a directory 1 above this one
#this dir contains a file called mem.py which contains MEMs
from Marginal_Effects_at_Means.mem import MEMs

In [274]:
all = trn.copy().append(tst.copy(), ignore_index=True)
all_y = trn_y.copy().append(tst_y.copy(), ignore_index=True)

In [275]:
all.head()

,cc.age,cc.blackdum,cc.cc16_304,cc.cc16_305_2,cc.cc16_422c,cc.cc16_422d,cc.cc16_422e,cc.cc16_422f,cc.catholic,cc.evanprot,cc.ideo7,cc.inddum,cc.religiosity,cc.repdum,cc.sex,cc.whitedum,cc.emp.nojob,cc.faminc,cc.i.white.educhs,cc.immviewsum,cc.maxeduc.4yr,cc.union,crashpc,demo.popdense,job.uer,mort.ucd.despair.disc95.pdpy,rustpc
0,0.689612,1,0.631982,0,0.517074,0.659101,0.818020,0.386066,0,0,0.225539,1,0.543035,0,1,0,0,1.124369,0,0.000000,0,0,0.954196,0.159130,0.767272,0.785942,0.911288
1,1.201895,0,0.947973,1,1.551223,0.988651,0.409010,1.158199,0,1,1.127693,1,1.267082,0,0,1,0,1.405461,0,1.275854,0,0,0.271458,0.047007,0.730736,1.020814,0.001882
2,1.103379,0,0.947973,0,0.517074,0.988651,1.636039,0.772133,0,0,0.902155,1,0.362024,0,0,1,0,0.983823,0,0.425285,1,0,0.201719,0.084833,0.507861,0.880143,0.255189
3,1.398927,0,0.631982,0,0.517074,0.329550,1.636039,0.386066,0,0,0.451077,1,1.086071,0,1,1,0,1.686553,0,0.000000,1,0,0.383675,0.040013,0.741697,1.058572,0.199546
4,1.261005,0,0.631982,0,1.034148,0.329550,0.409010,0.386066,0,0,0.676616,1,1.086071,0,0,1,0,1.264915,0,0.425285,0,0,0.069955,0.063948,0.801982,0.834638,0.000000


In [276]:
all.dtypes
len(all.columns)

cc.age                          float64
cc.blackdum                       int64
cc.cc16_304                     float64
cc.cc16_305_2                     int64
cc.cc16_422c                    float64
cc.cc16_422d                    float64
cc.cc16_422e                    float64
cc.cc16_422f                    float64
cc.catholic                       int64
cc.evanprot                       int64
cc.ideo7                        float64
cc.inddum                         int64
cc.religiosity                  float64
cc.repdum                         int64
cc.sex                            int64
cc.whitedum                       int64
cc.emp.nojob                      int64
cc.faminc                       float64
cc.i.white.educhs                 int64
cc.immviewsum                   float64
cc.maxeduc.4yr                    int64
cc.union                          int64
crashpc                         float64
demo.popdense                   float64
job.uer                         float64


27

In [277]:
#RUN ON WHOLE DATASET OR JUST THE TestSET?  I'm thinking the whole dataset.  
# The model has some idea of how voters will vote based on the input features, lets use that knowledge
#to see what happens when we start changing variables
mems = MEMs(all)

In [278]:
#whats the average look like?
mems.df_avg

,cc.age,cc.blackdum,cc.cc16_304,cc.cc16_305_2,cc.cc16_422c,cc.cc16_422d,cc.cc16_422e,cc.cc16_422f,cc.catholic,cc.evanprot,cc.ideo7,cc.inddum,cc.religiosity,cc.repdum,cc.sex,cc.whitedum,cc.emp.nojob,cc.faminc,cc.i.white.educhs,cc.immviewsum,cc.maxeduc.4yr,cc.union,crashpc,demo.popdense,job.uer,mort.ucd.despair.disc95.pdpy,rustpc
0,1.02961,0.0538659,0.999562,0.140415,0.942767,0.927069,0.835534,0.931723,0.197307,0.24512,0.931947,1,0.882436,0,0.438342,0.801937,0.0459979,0.977634,0.149342,0.85398,0.282796,0.307006,0.837256,0.189944,0.959738,0.973359,0.724066


# Results: 
- For each column
    - If binary column generate results on average row with that col set to min() and then max()
    - If continuous column generate results on average row with that col set to average and then 1 std deviation above average

In [279]:
#random forest
p=mems.getMEM_avg(m_rf)
p
p.shape
type(p)

array([[0.874371, 0.125629]])

(1, 2)

numpy.ndarray

In [280]:
def getMEM_CSV(m,mems,dep_var,df,type='_rf'):

    #The average prediction
    avpred = (mems.getMEM_avg(m))[0][1]

    #get all the columns we are going to run MEMs on
    cols = list(df.columns)
#     cols.remove(dep_var)#dump dependant variable

    #for binary values, predict when col=0 then predict when col=1, output is probability vote for depvar
    #NOTE one of the 2 results below will look like the average from above since the reality is that these values only
    #take 1 of 2 values. So the model will treat them as 1 of the 2, not an average of all columns.
    #for continuous values, predict when col=average then predict when col is +1 standard deviation away
    #output is probability vote for depvar

    #what goes in csv
    colsr = ['column_name','aveage_val','initval','initval_predict_prob','finalval','finalval_predict_prob', 'final_minus_init']
    df_res = pd.DataFrame(columns = colsr)

    for col in cols:
        p=mems.getMEM_avgplusoneSimple(m, col)
        df_res = df_res.append({colsr[0]:col,colsr[1]:float(mems.df_avg[col]),colsr[2]:p[0][0],colsr[3]:p[0][1],
                                colsr[4]:p[1][0],colsr[5]:p[1][1],colsr[6]:(p[1][1]-p[0][1])},ignore_index=True )

#     df_res.head()
    filename = outdir+ "/MEMS_" + dep_var +"_"+ filter +type+ ".csv"
    
    with open(filename, 'w') as f:
        f.write(f'Prediction for the average column is {avpred}')
        df_res.to_csv(f,encoding = "ISO-8859-1")               

In [281]:
#firest for rf
dftmp = all.copy()
getMEM_CSV(m_rf,mems,columns_dep_var,dftmp)

In [282]:
# a=outdir+ "/MEMS_" + columns_dep_var + filter +'_rf'+ ".csv"
# a

# See what the difference is between lr and rf

In [283]:
res_rf= pd.read_csv((outdir+ "/MEMS_" + dep_var +"_rf.csv"),encoding = "ISO-8859-1")
res_lr= pd.read_csv((outdir+ "/MEMS_" + dep_var +"_lr.csv"),encoding = "ISO-8859-1")

NameError: name 'dep_var' is not defined

In [ ]:
res_rf

In [ ]:
res_lr

In [ ]:
res_lr.head()
res_lr.describe()

In [ ]:
res_rf1 = res_rf[['column_name','final_minus_init']]
res_lr1 = res_lr[['column_name','final_minus_init']]


In [ ]:
res_rf1.describe()

In [ ]:
vals1 = pd.DataFrame(data=(res_rf1['column_name']))
f2 = (res_rf1['final_minus_init'] - res_lr1['final_minus_init']).to_frame()
# vals1.merge(((res_rf['final_minus_init'] - res_lr['final_minus_init'])).to_frame())

In [ ]:
vals1=pd.merge(vals1,f2,left_index=True, right_index=True);

In [ ]:
vals1.columns

In [ ]:
# vals1.merge(f2)
vals1.sort_values(by=['final_minus_init'], inplace=True)
vals1

In [ ]:
cols = ['cc.Ideo7','cc.Age','cc.CC16_304','cc.CC16_307','cc.raceviewsum','cc.IndDum','cc.Religiosity']
res_lr2 = res_lr[(res_lr['column_name'].isin(cols)) ]
res_rf2=res_rf[(res_lr['column_name'].isin(cols))]

cols_keep =['column_name','initval','initval_predict_prob','finalval','finalval_predict_prob','final_minus_init']
res_lr2[cols_keep]
res_rf2[cols_keep]

In [ ]:
# vals['column_name']=res_rf['column_name']

# len(vals)

# Scratch